In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image

###############################################################################
# 1) LOAD MODEL & PROCESSOR
###############################################################################
model_path = "qwen_model" 
processor = AutoProcessor.from_pretrained(model_path)
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    torch_dtype="auto",
    device_map="auto"  # Automatically distribute across available GPUs
)

###############################################################################
# 2) LOAD & PREPROCESS IMAGES
###############################################################################
# Example images (not passed to the model)
example_buyer_image_path = "COT/images/Buyer_image/0526.jpg"
example_seller_image_path = "COT/images/Seller_image/0526.jpg"

# Actual task images (passed to the model)
buyer_image_path = "COT/images/Buyer_image/0534.jpg"
seller_image_path = "COT/images/Seller_image/0534.jpg"

try:
    example_buyer_image = Image.open(example_buyer_image_path).convert("RGB").resize((448, 448))
    example_seller_image = Image.open(example_seller_image_path).convert("RGB").resize((448, 448))

    buyer_image = Image.open(buyer_image_path).convert("RGB").resize((448, 448))
    seller_image = Image.open(seller_image_path).convert("RGB").resize((448, 448))
except Exception as e:
    print(f"Error loading images: {e}")
    raise

###############################################################################
# 3) DEFINE TEXT INPUTS
###############################################################################
# Example case
example_buyer_description = "single shot example reveiw for buyer reveiw"
example_seller_description = (
    "sinlge shot example seller descrition"
)
example_response = (
    "The seller promises a high-quality watch with durable components. However, "
    "the buyer’s experience suggests a fault in the quality, as a link came undone "
    "within a few hours. This discrepancy indicates the product may not meet the "
    "durability standards implied by the seller’s description, making this an issue "
    "on the seller’s end."
)

# Actual case
buyer_description = (
    """buyer review
"""

)

seller_description = (
""" seller product description

"""
)

prompt = (
    """prompt for single shot """
)

###############################################################################
# 4) CREATE MESSAGES FOR SINGLE-SHOT EXAMPLE
###############################################################################
messages = [
    {
        "role": "user",
        "content": [
            # Example case
            {"type": "text", "text": "Example Case:"},
            {"type": "text", "text": f"Buyer Review: {example_buyer_description}"},
            {"type": "text", "text": f"Seller Description: {example_seller_description}"},
            {"type": "text", "text": f"Expected Output: {example_response}"},
            {"type": "text", "text": "---"},  # Separator

            # Actual case
            {"type": "text", "text": "Actual Task:"},
            {"type": "image", "image": buyer_image},
            {"type": "text", "text": f"Buyer Review: {buyer_description}"},
            {"type": "image", "image": seller_image},
            {"type": "text", "text": f"Seller Description: {seller_description}"},
            {"type": "text", "text": prompt},
        ],
    }
]

###############################################################################
# 5) GENERATE OUTPUT USING MODEL
###############################################################################
try:
    text_input = processor.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    if text_input is None:
        raise ValueError("Generated text from apply_chat_template is None.")

    inputs = processor(
        text=[text_input],
        images=[buyer_image, seller_image],
        return_tensors="pt",
        padding=True,
    ).to("cuda" if torch.cuda.is_available() else "cpu")

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=1500,
        temperature=0.7,
        repetition_penalty=1.2,
        top_k=50,
        top_p=0.9
    )

    output_text = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    print("Generated Structured Analysis:\n", output_text[0])

except Exception as e:
    print(f"Error during processing or inference: {e}")